In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
import numpy as np

In [15]:
will = pd.read_csv('data/derived/crime_ACS_merged.csv')
#all columns to numeric
for col in will.columns.values:
    will[col] = pd.to_numeric(will[col], errors='coerce')

#TODO: un-hackify this
#drop columns with many NaN
bad_cols = will.isnull().sum()>100
will = will.loc[:,~bad_cols]

np.sum(will.isnull().sum(axis=1)==0)

61

In [10]:
x_data=will[will.columns.difference(['Murder and nonnegligent manslaughter'])]
y_data=will['Murder and nonnegligent manslaughter']

train_x,test_x, train_y,test_y = train_test_split(x_data, y_data, test_size=0.3, random_state=42)


In [11]:
fitted_model = LassoCV().fit(train_x, train_y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').